1. 문서의 내용을 읽는다 (docx2txt)
2. 문서를 쪼갠다 (langchain-text-splitters)
  - 토큰수 초과로 답변을 생성하지 못할수 있고
  - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장 (OpenAIEmbeddings, chroma)
4. 질문이 있을 때, 백터 테이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 (similarity_search)

In [ ]:
%pip install --upgrade --quiet  docx2txt langchain-community -qU langchain-text-splitters

In [5]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500, # 하나의 청크가 가질수있는 토큰수
  chunk_overlap=200 # 청크 간 중복 토큰수
)

loader = Docx2txtLoader("./tax.docx")

document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
len(document_list)

In [7]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [3]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

NameError: name 'document_list' is not defined

In [ ]:
%pip install langchain-chroma

In [9]:
from langchain_chroma import Chroma

# ./chroma에 저장된 내용 가져와서 필요없는건 주석

#database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma_tax" persist_directory="./chroma")
database = Chroma(collection_name="chroma_tax", persist_directory="./chroma", embedding_function=embedding)

In [ ]:
%pip install -qU langchain langchain-pinecone langchain-openai

In [10]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#retrieved_docs = database.similarity_search(query, k=3) 

In [ ]:
# retrieved_docs

In [47]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')


In [48]:
# prompt = f"""[Identity]
# - 당신은 최고의 소득세 전문가 입니다
# -[Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# Question: {query}
# """

In [49]:
# ai_message = llm.invoke(prompt)

In [ ]:
ai_message.content

In [ ]:
%pip install -U langchain langchainhub --quiet

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

In [54]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=database.as_retriever(),
  chain_type_kwargs={"prompt": prompt}
)

In [56]:
ai_message = qa_chain({"query": query})

In [ ]:
ai_message